In [117]:
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-06-23T21:57:09.203197-04:00

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

Compiler    : MSC v.1937 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 167 Stepping 1, GenuineIntel
CPU cores   : 16
Architecture: 64bit



In [118]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import xgboost as xgb

pd.set_option('display.float_format', lambda x: '%.3f' % x)

%matplotlib inline
%watermark -w
%watermark -iv

Watermark: 2.4.3

seaborn   : 0.13.2
matplotlib: 3.8.3
numpy     : 1.26.4
geopandas : 0.14.3
pandas    : 2.2.1
xgboost   : 2.1.0



In [119]:
sns.set_style('ticks')

warnings.filterwarnings('ignore')


seed = 1234
np.random.seed(seed)
random.seed(seed)

## Funcoes de Apoio

In [120]:
# Separar os dados em treino e em teste

def selecao_teste_treino(dados, test_ratio):
    indices_aleat = np.random.permutation(len(dados))
    tamanho_base = int(len(dados) * test_ratio)
    indices_test = indices_aleat[:tamanho_base]
    indices_treino = indices_aleat[tamanho_base:]
    return dados.iloc[indices_treino], dados.iloc[indices_test]

In [121]:
# AJUSTE DAS COLUNAS SITUACAO DE CONCLUSAO E ANO DE CONCLUSAO

def ajuste_var_ano_conclusao(linha, coluna1, coluna2):
    """
    Ajusta a variável de ano de conclusão com base nas colunas especificadas.

    Parâmetros:
    dados (DataFrame): O DataFrame contendo os dados.
    coluna1 (str): O nome da primeira coluna para verificar.
    coluna2 (str): O nome da segunda coluna para verificar.

    Retorna:
    int: O ano ajustado de conclusão.
    """

    mapping_coluna2 = {
        1: 2022,
        2: 2021,
        3: 2020,
        4: 2019,
        5: 2018,
        6: 2017,
        7: 2016,
        8: 2015,
        9: 2014,
        10: 2013,
        11: 2012,
        12: 2011,
        13: 2010,
        14: 2009,
        15: 2008,
        16: 2007,
        17: 2006
    }



    if linha[coluna2] == 0 and linha[coluna1] == 2:
        return 2023
    elif linha[coluna2] == 0 and linha[coluna1] == 3:
        return 2024
    elif linha[coluna2] == 0 and linha[coluna1] == 4:
        return 2025 # são apenas 10.517 casos
    elif linha[coluna2] == 0 and linha[coluna1] == 1:
        return 2006 # são 122.901 casos de pessoas que já concluíram e não informaram o ano
    elif linha[coluna2] != 0:
        return mapping_coluna2.get(linha[coluna2], linha[coluna2])
    else:
        return linha[coluna2]

## Dados

In [122]:
# Data

data = pd.read_csv("microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv", encoding='latin', sep = ';')

In [123]:
colunas1 = ['NU_INSCRICAO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
            'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
           'IN_TREINEIRO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
           ]

colunas2 = data.loc[:,'Q001':'Q025'].columns.to_list()
colunas3 = colunas1 + colunas2

dados_modelo = data[colunas3]

In [124]:
def faltou(coluna1):
    if coluna1 == 1 or coluna1 == 2:
        return 'present'
    else:
        return 'falta'

dados_modelo['target'] = dados_modelo['TP_PRESENCA_CN'].apply(faltou)

dados_modelo.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_CH,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,target
0,210059085136,14,M,2,1,1,17,0,0,0,...,C,B,B,A,B,B,A,A,B,falta
1,210059527735,12,M,2,1,1,16,0,0,0,...,A,B,B,A,A,C,A,D,B,falta
2,210061103945,6,F,1,1,1,0,0,1,1,...,A,A,B,A,A,A,A,A,B,present
3,210060214087,2,F,1,3,2,0,0,1,1,...,A,A,B,A,A,D,A,A,B,present
4,210059980948,3,F,1,3,2,0,0,1,1,...,A,A,B,A,A,B,A,A,A,present


In [125]:
# Separar os dados em treino e em teste

dados_treino, dados_teste = selecao_teste_treino(dados_modelo, 0.4)

In [126]:
print(f'A base treino possui {len(dados_treino)} registros.')
print(f'A base teste possui {len(dados_teste)} registros.')
print(f'A proporcao de resgistros de treino e de {len(dados_treino)/len(dados_modelo)*100}%.')

A base treino possui 2360373 registros.
A base teste possui 1573582 registros.
A proporcao de resgistros de treino e de 60.0%.


In [127]:
# Avaliando a proporcao da variavel target dentro de cada base de dados.
# Essa e uma pratica que eu gosto de adotar quando realizo a separacao das bases, apenas como um criterio de validacao.

print(f'A proporcao de presentes na base treino e de {round(dados_treino['target'].value_counts()[0]/dados_treino['target'].value_counts().sum()*100,5)}%.')
print(f'A proporcao de presentes na base teste e de {round(dados_teste['target'].value_counts()[0]/dados_teste['target'].value_counts().sum()*100,5)}%.')

A proporcao de presentes na base treino e de 68.49811%.
A proporcao de presentes na base teste e de 68.49519%.


In [128]:
caracteristicas_participantes = dados_treino.drop('target', axis = 1)
labels = dados_treino['target'].copy()

In [129]:
# Removendo as colunas com a informacao de presenca x ausencia

caracteristicas_participantes = caracteristicas_participantes.drop(['NU_INSCRICAO','TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',], axis=1)

## Tratamentos

### Regras Adotadas:
- Criacao de uma nova variavel para: TP_ST_CONCLUSAO e TP_ANO_CONCLUIU: foi criada uma nova variavel acima contemplando as duas variaveis
- Remocao da var IN_TREINEIRO: todos os participantes inscritos como treineiros possuem a a classificacao: "Estou cursando e concluirei o Ensino Médio após 2023 em TP_ST_CONCLUSAO".
  
- Qualitativas Ordinais:
  - TP_FAIXA_ETARIA (ja codificado na origem)
  - Q001 e Q002
  - Q003 e Q004 (sera considerada ordinal pois e possivel extrair uma hierarquia nessas profissoes)
  - Q006
  - Q007
  - Q008
  - Q009
  - Q010
  - Q011
  - Q012
  - Q013
  - Q014
  - Q015
  - Q016
  - Q017
  - Q019
  - Q022
  - Q024
  
- Qualitativas: 
  - 2 VAR:
    - TP_SEXO (considerar 0 e 1)
    - Q018 (considerar 0 e 1)
    - Q020 (considerar 0 e 1)
    - Q021 (considerar 0 e 1)
    - Q024 (considerar 0 e 1)
    - Q025 (considerar 0 e 1)
  
  - + 2 VARS:
    - TP_ESTADO_CIVIL
    - TP_COR_RACA
 


- Pensando:
  - Considerar casas com mais de 10 pessoas morando como 10.

In [130]:
# Ajuste das variaveis ano e data de conclusao

caracteristicas_participantes['ANO_CONCLUSAO_AJUSTADO'] = caracteristicas_participantes.apply(ajuste_var_ano_conclusao, axis=1,
                                                                                   coluna1 = 'TP_ST_CONCLUSAO', coluna2 = 'TP_ANO_CONCLUIU')

In [131]:
# Remocao das variaveis:
# TP_ST_CONCLUSAO e TP_ANO_CONCLUIU: foi criada uma nova variavel acima contemplando as duas variaveis
# IN_TREINEIRO: todos os participantes inscritos como treineiros possuem a a classificacao: Estou cursando e concluirei o Ensino Médio após 2023 em TP_ST_CONCLUSAO

caracteristicas_participantes = caracteristicas_participantes.drop(['TP_ANO_CONCLUIU', 'IN_TREINEIRO', 'TP_ST_CONCLUSAO'], axis=1)

In [132]:
# TP_SEXO

encoder = LabelEncoder()

caracteristicas_participantes['TP_SEXO'] = encoder.fit_transform(caracteristicas_participantes['TP_SEXO'])
caracteristicas_participantes['Q018'] = encoder.fit_transform(caracteristicas_participantes['Q018'])
caracteristicas_participantes['Q020'] = encoder.fit_transform(caracteristicas_participantes['Q020'])
caracteristicas_participantes['Q021'] = encoder.fit_transform(caracteristicas_participantes['Q021'])


In [133]:
display(caracteristicas_participantes)

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,Q001,Q002,Q003,Q004,Q005,Q006,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,ANO_CONCLUSAO_AJUSTADO
2143670,2,0,1,1,C,E,F,B,4,G,...,A,0,C,1,0,E,A,B,B,2023
2983198,2,0,1,3,E,G,D,D,4,K,...,A,1,E,0,0,E,A,E,B,2023
968989,11,0,1,1,D,D,D,B,4,G,...,A,1,C,0,1,D,A,B,B,2013
1659500,3,1,1,3,E,E,F,F,4,B,...,A,0,B,0,0,C,A,B,B,2023
2097665,4,1,1,2,C,B,C,C,3,E,...,A,0,B,1,1,D,A,B,B,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978124,10,0,1,3,B,D,B,B,2,B,...,A,0,B,0,0,B,A,A,B,2025
1956917,9,0,1,3,H,E,F,B,3,B,...,A,0,C,0,0,C,B,B,B,2016
165158,2,0,1,1,C,B,A,A,3,D,...,A,1,C,0,0,D,A,C,B,2023
2548435,5,1,1,2,D,E,F,B,2,C,...,A,1,A,0,0,C,A,B,B,2021


In [134]:
## TP_ESTADO_CIVIL

mapa_estado_civil = {
    0: 'NA_INF',
    1: 'SOLT',
    2: 'CASAD',
    3: 'SEPAR',
    4: 'VIUVO'
}

caracteristicas_participantes['TP_ESTADO_CIVIL'] = caracteristicas_participantes['TP_ESTADO_CIVIL'].map(mapa_estado_civil)

In [135]:
caracteristicas_participantes

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,Q001,Q002,Q003,Q004,Q005,Q006,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,ANO_CONCLUSAO_AJUSTADO
2143670,2,0,SOLT,1,C,E,F,B,4,G,...,A,0,C,1,0,E,A,B,B,2023
2983198,2,0,SOLT,3,E,G,D,D,4,K,...,A,1,E,0,0,E,A,E,B,2023
968989,11,0,SOLT,1,D,D,D,B,4,G,...,A,1,C,0,1,D,A,B,B,2013
1659500,3,1,SOLT,3,E,E,F,F,4,B,...,A,0,B,0,0,C,A,B,B,2023
2097665,4,1,SOLT,2,C,B,C,C,3,E,...,A,0,B,1,1,D,A,B,B,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978124,10,0,SOLT,3,B,D,B,B,2,B,...,A,0,B,0,0,B,A,A,B,2025
1956917,9,0,SOLT,3,H,E,F,B,3,B,...,A,0,C,0,0,C,B,B,B,2016
165158,2,0,SOLT,1,C,B,A,A,3,D,...,A,1,C,0,0,D,A,C,B,2023
2548435,5,1,SOLT,2,D,E,F,B,2,C,...,A,1,A,0,0,C,A,B,B,2021


In [158]:
99999999999999999999999999999999999999999999999999999999999999999999999999999999



KeyError: "None of [Index(['TP_ESTADO_CIVIL'], dtype='object')] are in the [columns]"

In [155]:
df_onehot

,TP_ESTADO_CIVIL_CASAD,TP_ESTADO_CIVIL_NA_INF,TP_ESTADO_CIVIL_SEPAR,TP_ESTADO_CIVIL_SOLT,TP_ESTADO_CIVIL_VIUVO
0,0.000,0.000,0.000,1.000,0.000
1,0.000,0.000,0.000,1.000,0.000
2,0.000,0.000,0.000,1.000,0.000
3,0.000,0.000,0.000,1.000,0.000
4,0.000,0.000,0.000,1.000,0.000
...,...,...,...,...,...
2360368,0.000,0.000,0.000,1.000,0.000
2360369,0.000,0.000,0.000,1.000,0.000
2360370,0.000,0.000,0.000,1.000,0.000
2360371,0.000,0.000,0.000,1.000,0.000


In [145]:
onehot_encoder.get_feature_names_out()

array(['TP_ESTADO_CIVIL_CASAD', 'TP_ESTADO_CIVIL_NA_INF',
       'TP_ESTADO_CIVIL_SEPAR', 'TP_ESTADO_CIVIL_SOLT',
       'TP_ESTADO_CIVIL_VIUVO'], dtype=object)